## Finetuning Event Detection on TAK

In [2]:
import pandas as pd
import phospho 
from dotenv import load_dotenv


In [3]:
load_dotenv()

phospho.init()
tasks_df = phospho.tasks_df(limit=10_000)
tasks_df.set_index("task_id", inplace=True)
tasks_df

,task_input,task_output,task_metadata,task_eval,task_eval_source,task_eval_at,task_created_at,session_id,session_length,event_name,event_created_at
task_id,,,,,,,,,,,
df4eaba05f16454d824d7bbc4c495fbd,Combien y a t il de camions en Europe ?,"En 2024, il y a plus de 6,5 millions de camion...","{'version_id': 'gpt-4', 'client_created_at': 1...",success,owner,2024-05-29 15:55:11,2024-05-24 18:04:03,Sv6NeyX-the-tak,1,None,NaT
8e99109451be48e5a76ca4b262707b58,Que peux tu faire ?,"Je suis Tak, un expert en recherche web créé p...","{'version_id': 'gpt-4', 'client_created_at': 1...",success,phospho-4,2024-05-24 18:11:07,2024-05-24 18:03:23,z5qgRtM-the-tak,1,phospho,NaT
1e726a58c8254bb8aa6a0c205eb5c7ed,Dernières nouvelles sur l'IA en France,Voici les dernières nouvelles sur l'intelligen...,"{'version_id': 'gpt-4', 'client_created_at': 1...",success,phospho-4,2024-05-24 18:11:08,2024-05-24 17:13:57,LZVP3cV-the-tak,1,news,NaT
1e726a58c8254bb8aa6a0c205eb5c7ed,Dernières nouvelles sur l'IA en France,Voici les dernières nouvelles sur l'intelligen...,"{'version_id': 'gpt-4', 'client_created_at': 1...",success,phospho-4,2024-05-24 18:11:08,2024-05-24 17:13:57,LZVP3cV-the-tak,1,AI,NaT
fdf732ca4c2f47f681e0730c6a41b69a,Écris moi un poème sur les robots,"Bien sûr, voici un poème sur les robots :\n\n-...","{'version_id': 'gpt-4', 'client_created_at': 1...",success,phospho-4,2024-05-24 18:11:09,2024-05-24 17:13:33,DVP9VqE-the-tak,1,None,NaT
...,...,...,...,...,...,...,...,...,...,...,...
41248135de364ac294cfb0393f5f67bf,how to abolish capitalism ?,None,"{'client_created_at': 1715288110, 'last_update...",success,owner,2024-05-25 07:17:44,2024-05-09 20:55:10,SAO979YT1D,1,None,NaT
99275f11cd6148079a3600ec0231363f,How to use tqm in python?,None,"{'client_created_at': 1715269690, 'last_update...",success,owner,2024-05-25 07:17:45,2024-05-09 15:48:10,J8SGV27O91,1,code,NaT
573a58cdfa9c4f4c8f00a4a46e2dc5bf,Write me a poem about robots,None,"{'client_created_at': 1715269012, 'last_update...",success,owner,2024-05-25 07:32:01,2024-05-09 15:36:52,GLK6WEFS10,1,None,NaT


In [18]:
new_df = tasks_df.dropna(subset=["task_input", "task_output"]).copy()
new_df["text"] = new_df[['task_input', 'task_output']].agg(' '.join, axis=1)

event_df = pd.get_dummies(tasks_df['event_name'])
event_df = event_df.astype(int)
new_df = new_df[['text']].merge(event_df, left_index=True, right_index=True)

new_df = new_df.drop_duplicates()
new_df = new_df.groupby(by=["task_id", "text"]).sum().reset_index()
new_df = new_df.set_index("task_id")

new_df

,text,AI,code,news,phospho
task_id,,,,,
00362cd6be974fffacc48b4690481216,que faire a new york ce week end? Voici quelq...,0,0,0,0
007452c5b9d242f48254fcf348a96fa6,"Réécris la chanson Gigi l'amoroso, en décrivan...",0,0,0,0
0122446bf08645b9971e1ebf7cfa942f,this doesn't create a nested structure You're ...,0,1,0,0
01ccbfd37d014d6d8d90ba39b2a202ed,la pratique du trail et de l'ultra trail provo...,0,0,0,0
01d135ffb53c4402b22684fa8abb6872,Quelle est la proportion d'étudiants ingénieur...,0,0,0,0
...,...,...,...,...,...
fee8ca4afcc2427f81b9052a4b3f73a0,Fais moi un plan détaillé d'un roadtrip de 4 m...,0,0,0,0
fef89e16e22943e8a6bef2acbc6cd957,quelle particularité as tu par rapport a chatG...,0,0,0,1
ff5acf321e27424ca1a6b408cd85c86e,make a judicial review for the massachusetts v...,0,0,0,0


In [5]:
new_df[["AI", "code", "news", "phospho"]].sum(axis=1).max()

3

# A first baseline model

In [6]:
!pip install x-transformers

In [11]:
from sklearn.model_selection import train_test_split

X = new_df['text']
y = new_df.drop(columns="text")
 
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.15, random_state=42)

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import f1_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb

class_weights = {0: 1, 1: 10}  # Class 0: 1, Class 1: 10
# Set "scale_pos_weight" based on class balance
pos_weight = sum(class_weights.values()) / sum(class_weights.keys())


text_classifier = make_pipeline(
    TfidfVectorizer(min_df=3, max_df=0.8, ngram_range=(1, 2)),
    MultiOutputClassifier(estimator= xgb.XGBClassifier(scale_pos_weight=pos_weight, class_weight=class_weights))
)

%time _ = text_classifier.fit(X_train, y_train)
y_pred = text_classifier.predict(X_test)
f1_score(y_test, y_pred, average="weighted")


/Users/kyoto/miniforge3/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [17:03:37] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "class_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/kyoto/miniforge3/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [17:03:38] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "class_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/kyoto/miniforge3/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [17:03:39] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "class_weight" } are not used.

  warnings.warn(smsg, UserWarning)


CPU times: user 14.8 s, sys: 6.05 s, total: 20.8 s
Wall time: 2.61 s


0.8853881158198047

In [9]:
preds_data = text_classifier.predict(new_df["text"])
preds = pd.DataFrame(
    columns=[f"{col}_pred" for col in y_train.columns], 
    data=preds_data,
    index=new_df.index,
)
preds_df = pd.concat([new_df, preds], axis=1)
preds_df

,text,AI,code,news,phospho,AI_pred,code_pred,news_pred,phospho_pred
task_id,,,,,,,,,
00362cd6be974fffacc48b4690481216,que faire a new york ce week end? Voici quelq...,0,0,0,0,0,0,0,0
007452c5b9d242f48254fcf348a96fa6,"Réécris la chanson Gigi l'amoroso, en décrivan...",0,0,0,0,0,0,0,0
0122446bf08645b9971e1ebf7cfa942f,this doesn't create a nested structure You're ...,0,1,0,0,0,1,0,0
01ccbfd37d014d6d8d90ba39b2a202ed,la pratique du trail et de l'ultra trail provo...,0,0,0,0,0,0,0,0
01d135ffb53c4402b22684fa8abb6872,Quelle est la proportion d'étudiants ingénieur...,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
fee8ca4afcc2427f81b9052a4b3f73a0,Fais moi un plan détaillé d'un roadtrip de 4 m...,0,0,0,0,0,0,0,0
fef89e16e22943e8a6bef2acbc6cd957,quelle particularité as tu par rapport a chatG...,0,0,0,1,0,0,0,1
ff5acf321e27424ca1a6b408cd85c86e,make a judicial review for the massachusetts v...,0,0,0,0,0,0,0,0


In [10]:
is_bad_pred = (y_pred != y_test)['news']
is_bad_pred.name = "is_bad_pred_code"

bad_code_preds = preds_df.merge(is_bad_pred, left_index=True, right_index=True)
bad_code_preds

,text,AI,code,news,phospho,AI_pred,code_pred,news_pred,phospho_pred,is_bad_pred_code
task_id,,,,,,,,,,
01ccbfd37d014d6d8d90ba39b2a202ed,la pratique du trail et de l'ultra trail provo...,0,0,0,0,0,0,0,0,False
02f59fc6aac34ca38e56474e97074432,transform this note in a Guideline process or ...,0,0,0,0,0,0,0,0,False
05199b2cb40e446e8ba0f851d9155f89,give me a beam emoji Here is a beam emoji for ...,0,0,0,0,0,0,0,0,False
07b7c70f9422416e8546bece550056d0,Write me a poem about robots **Ode to the Meta...,0,0,0,0,0,0,0,0,False
08b659c4a1454b868b7333b6ebe42c36,What shows that Spain emphasizes the importanc...,0,0,1,0,0,0,1,0,False
...,...,...,...,...,...,...,...,...,...,...
fb8bb8cd3027488cbc3b0d4b1c554a4c,Ferme solaire de Crucey\t\tEure-et-Loir\tUrbas...,0,0,0,0,0,0,0,0,False
fdf732ca4c2f47f681e0730c6a41b69a,"Écris moi un poème sur les robots Bien sûr, vo...",0,0,0,0,0,0,0,0,False
fe9bd150b5e943178bb6c1f5add9d8fb,Dernières nouvelles sur l'IA en France Voici l...,1,0,1,0,1,0,1,0,False


### Baseline Test Scores

- code: 0.9
- news: 0.85
- phospho: 0.98
- AI: 0.86